<a href="https://colab.research.google.com/github/Ahmed-Hossam-Aldeen/Slowed-Reverb/blob/main/Slow_and_Reverb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install youtube_dl
!pip install pydub
!pip install simple_youtube_api
!pip install pysndfx
!pip install glitch_this

In [ ]:
!apt -qq install -y sox

In [7]:
#SLOWED---WITH---LINK
from __future__ import unicode_literals
import youtube_dl
import os
from scipy.io import wavfile
import wave
import os
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment
import logging
import librosa as lr
import soundfile as sf
from pysndfx.dsp import AudioEffectsChain

link = input("Enter Link: ")
speed = input("Enter Speed: ")
upload = "y"
color = "red"
ydl_opts = {
    #'outtmpl': '/' + str(input) +'.%(mp3)s',
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '320',

    }],
}

with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    info_dict = ydl.extract_info(link, download=False)
    title = info_dict.get('title', None)
    id=video_id = info_dict.get("id", None)
    ful= title+' '+"[slowed + reverb]"+'.'+'mp3'
    con= title+'-'+id+'.'+'mp3'
    conc=con.replace("|","_").replace("/","_")
    full=ful.replace("|","_").replace("/","_")
    print(conc)
    print(full)
    inputt=title+'.'+'mp3'
    inputt= inputt.replace("|","_").replace("/","_")
    ydl.download([link])
os.rename('./%s'%conc, "input.mp3")

#------------------------------------------------------------------------------#
 #Define conversion format function
def trans_mp3_to_wav(filepath):
    song = AudioSegment.from_mp3(filepath)
    song.export("temp.wav", format="wav")
trans_mp3_to_wav("./input.mp3")


logger = logging.getLogger('pysndfx')
logger.setLevel(logging.DEBUG)
apply_audio_effects =     (AudioEffectsChain()
     .speed(speed)
     .reverb()
    )
infile = './temp.wav'
mono, sr = lr.load(infile, sr=None)
stereo, _ = lr.load(infile, sr=None, mono=False)
outfile = 'output.wav'

def test_file_to_file():
    apply_audio_effects(infile, outfile)
    y = lr.load(outfile, sr=None, mono=False)[0]
    sf.write('output.wav', y.T, sr)
    assert lr.util.valid_audio(y, mono=False)
test_file_to_file()
os.remove('./temp.wav')

import pydub
sound = pydub.AudioSegment.from_wav("./output.wav")
sound.export("final.mp3", format="mp3")
os.remove('./output.wav')

os.rename("final.mp3","%s"%full)
os.rename("input.mp3","%s"%inputt)
################################################################################
if upload == "y":
  from simple_youtube_api.Channel import Channel
  from simple_youtube_api.LocalVideo import LocalVideo
  import os
  import requests # to get image from the web
  import shutil # to save it locally

  name = "%s" %full
  name = name.replace(".mp3","")
  if len(name)>70:
    name=name[0:42]+" [slowed + reverb]"
  print(name)
  os.system(f"youtube-dl --write-thumbnail --skip-download --output img {link}")
  if os.path.isfile('img.png'):
    os.rename("img.png","img.jpg")
  elif os.path.isfile('img.webp'):
    os.rename("img.webp","img.jpg")

  import PIL
  from PIL import Image 
  img = PIL.Image.open("img.jpg")
  width, height = img.size
  if(width<1000 and height <600):
  ###################################################################################################
    ## Set up the image URL and filename
    image_url = input ("if Low quality Found, Enter new image link :") 
    filename = 'img.jpg'
    # Open the url image, set stream to True, this will return the stream content.
    r = requests.get(image_url, stream = True)
    # Check if the image was retrieved successfully
    if r.status_code == 200:
        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        r.raw.decode_content = True

        # Open a local file with wb ( write binary ) permission.
        with open(filename,'wb') as f:
            shutil.copyfileobj(r.raw, f)

        print('Image sucessfully Downloaded: ',filename)
    else:
        print('Image Couldn\'t be retreived')
  #########################################################################################################
  !glitch_this img.jpg 2.2 -s -c -f

  full2 = "'"+full+"'"
  command = (f'ffmpeg -i {full2} -i glitched_img.jpg -filter_complex "[0:a]aformat=channel_layouts=mono,showwaves=s=1280x202:mode=line:colors={color}|black[sw];[1][sw]overlay=(W-w)/2:(H-h)/2:format=auto,format=yuv420p[v]" -map "[v]" -map 0:a -movflags +faststart videoMono.mp4')
  os.system(command)

  # loggin into the channel
  channel = Channel()
  channel.login("./drive/MyDrive/0Slowed files/client_secret.json", "./drive/MyDrive/0Slowed files/credentials.storage")

  # setting up the video that is going to be uploaded
  video = LocalVideo(file_path="videoMono.mp4")

  # setting snippet
  video.set_title(name)
  video.set_description("I don't own music in this video, Uploaded using youtube API and Google Collab")
  video.set_tags(["this", "tag"])
  video.set_category("music")
  video.set_default_language("en-US")

  # setting status
  video.set_embeddable(True)
  video.set_license("creativeCommon")
  video.set_privacy_status("public")
  video.set_public_stats_viewable(True)

  # setting thumbnail
  video.set_thumbnail_path('glitched_img.jpg')

  # uploading video and printing the results
  video = channel.upload_video(video)
  print(video.id)
  print(video)

  # liking video
  video.like()

  os.remove('./videoMono.mp4')

Enter Link: https://www.youtube.com/watch?v=l-i3GBBeBZ4
Enter Speed: 0.8
[youtube] l-i3GBBeBZ4: Downloading webpage
Abyusif - 8_8_G (Prod. Abyusif)-l-i3GBBeBZ4.mp3
Abyusif - 8_8_G (Prod. Abyusif) [slowed + reverb].mp3
[youtube] l-i3GBBeBZ4: Downloading webpage
[download] Destination: Abyusif - 8_8_G (Prod. Abyusif)-l-i3GBBeBZ4.webm
[download] 100% of 3.05MiB in 00:48
[ffmpeg] Destination: Abyusif - 8_8_G (Prod. Abyusif)-l-i3GBBeBZ4.mp3
Deleting original file Abyusif - 8_8_G (Prod. Abyusif)-l-i3GBBeBZ4.webm (pass -k to keep)
Abyusif - 8_8_G (Prod. Abyusif) [slowed + reverb]
Glitched Image saved in "glitched_img.jpg"
Time taken to glitch: 0.05449175834655762
Time taken to save: 0.030420303344726562
Total Time taken: 0.08491253852844238
Current version: 1.0.2 | Latest version: 1.0.2


Upload:   0% |/                                      | ETA:  --:--:--   0.0 s/B

Video size: 21333315 bytes


Upload: 100% ||                                      | Time:  0:00:03 316.8 B/s


hEr-O4VXPg0
Title: 
Description: 
 Tags:[]


In [ ]:
from simple_youtube_api.Channel import Channel
from simple_youtube_api.LocalVideo import LocalVideo
import os
import requests # to get image from the web
import shutil # to save it locally

name = "%s" %full
print(name)
os.system(f"youtube-dl --write-thumbnail --skip-download --output img {link}")
if os.path.isfile('img.png'):
 os.rename("img.png","img.jpg")
elif os.path.isfile('img.webp'):
 os.rename("img.webp","img.jpg")

###################################################################################################
# ## Set up the image URL and filename
# image_url = "https://i.pinimg.com/originals/94/cb/45/94cb454c16bcc05fba36e0e7444488a1.png"
# filename = 'img.jpg'
# # Open the url image, set stream to True, this will return the stream content.
# r = requests.get(image_url, stream = True)
# # Check if the image was retrieved successfully
# if r.status_code == 200:
#     # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
#     r.raw.decode_content = True

#     # Open a local file with wb ( write binary ) permission.
#     with open(filename,'wb') as f:
#         shutil.copyfileobj(r.raw, f)

#     print('Image sucessfully Downloaded: ',filename)
# else:
#     print('Image Couldn\'t be retreived')
#########################################################################################################
!glitch_this img.jpg 2.2 -s -c -f

full2 = "'"+full+"'"
command = (f'ffmpeg -i {full2} -i img.jpg -filter_complex "[0:a]aformat=channel_layouts=mono,showwaves=s=1280x202:mode=line:colors=red|black[sw];[1][sw]overlay=(W-w)/2:(H-h)/2:format=auto,format=yuv420p[v]" -map "[v]" -map 0:a -movflags +faststart videoMono.mp4')
os.system(command)

# loggin into the channel
channel = Channel()
channel.login("./drive/MyDrive/0Slowed files/client_secret.json", "./drive/MyDrive/0Slowed files/credentials.storage")

# setting up the video that is going to be uploaded
video = LocalVideo(file_path="videoMono.mp4")

# setting snippet
video.set_title(name)
video.set_description("I don't own music in this video, Uploaded using youtube API and Google Collab")
video.set_tags(["this", "tag"])
video.set_category("music")
video.set_default_language("en-US")

# setting status
video.set_embeddable(True)
video.set_license("creativeCommon")
video.set_privacy_status("public")
video.set_public_stats_viewable(True)

# setting thumbnail
video.set_thumbnail_path('glitched_img.jpg')

# uploading video and printing the results
video = channel.upload_video(video)
print(video.id)
print(video)

# liking video
video.like()

os.remove('./videoMono.mp4')

In [ ]:
import os

print(full)
color = "black"
full2 = "'"+full+"'"
command = (f'ffmpeg -i {full2} -i img.jpg -filter_complex "[0:a]aformat=channel_layouts=mono,showwaves=s=1280x202:mode=line:colors={color}|black[sw];[1][sw]overlay=(W-w)/2:(H-h)/2:format=auto,format=yuv420p[v]" -map "[v]" -map 0:a -movflags +faststart videoMono.mp4')
#os.system(command)
print(command)

In [ ]:
# importing the module
import PIL
from PIL import Image 
img = PIL.Image.open("img.jpg")
width, height = img.size
if(width<1200 and height <720):
  print(img.size)
    

(336, 188)


In [ ]:
idk = "Abo El Anwar X Lil Baba- Yemken (Official music video) l ليل بابا - يمكن Xأبو الأنوار [slowed + reverb]"
idk = idk[0:40]+" [slowed + reverb]"
print(len(idk))
print(idk)


58
Abo El Anwar X Lil Baba- Yemken (Officia [slowed + reverb]
